# 05_validation_25D_test_5fold.ipynb

This script contain the validation process of the 2.5D 5-fold cross validation model on the independent test set. It generates the result on the publication. Operating point (obtained from the training and validation sets) is applied to the final classification on the CT scans. 

Required :
- `test_5fold_df.csv` (generated from `03_model_25D_5fold.ipynb`).
- `operating_point.out` (generated from `04_operatingpoint_trainval_25D_5fold.ipynb`)

Generate:
- the evaluation matrix of the independent test set, including:
    - the confusion matrix
    - the ROC curve
    - accuracy, sensitivity, specificity, precision, PPV, and NPV
    - 95% confidence interval (on 2.5D images).

In [ ]:
import sys
repo_dir = <PATH TO THIS REPO> # local path to this repo.
sys.path.insert(0, repo_dir)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import math

pd.set_option("display.max_columns", 20)
pd.set_option("display.max_colwidth", 20)

# utilitiy files located in the src folder
from src.util_analysis import analysis_1fold, analysis_1fold_95CT
from src.util_analysis import get_df_image_performance

In [2]:
# load the value of the operating point
op = np.loadtxt('operating_point.out').tolist()
# load the model predictions of the test set
test_df = pd.read_csv("test_5fold_df.csv") 
# get 2.5D slice number w.r.t the CT scans
slicenumber_list = []
for i in range(len(test_df)):
    slicenumber_list.append(int((os.path.splitext(test_df.jpeg_name[i])[0]).split("_")[1]))
test_df["slicenumber"] = slicenumber_list
df = test_df.sort_values(["PatientName","image_name","slicenumber"], ascending=[True, True, True]).reset_index(drop=True)

### Make predictions by combining the 5 models predictions (from the 5-fold cross validation)

In [3]:
# 5-fold cross validation combined prediction
df["test_prediction_5fold"] = df.apply(lambda x: np.average([x.test_prediction_model_fold1, x.test_prediction_model_fold2, x.test_prediction_model_fold3, x.test_prediction_model_fold4, x.test_prediction_model_fold5]), axis=1)
df["predicted_classes_avg_5fold"] = df.apply(lambda x: np.average([x.predicted_classes_model_fold1, x.predicted_classes_model_fold2, x.predicted_classes_model_fold3, x.predicted_classes_model_fold4, x.predicted_classes_model_fold5]), axis=1)
df["predicted_classes_5fold"] = np.where(df["predicted_classes_avg_5fold"]<0.5,0,1)

### Analysing the 5-fold prediction on the test set (in the unit of 2.5D images).

With 95% confidence interval. 

In [4]:
# load predictions from the DataFrame in the unit of 2.5D images
predicted_classes_5fold = np.array(df.predicted_classes_5fold, dtype=int)
test_prediction_5fold = np.array(df.test_prediction_5fold, dtype=float)
# load ground truth from the DataFrame in the unit of 2.5D images
true_classes = np.array(df.abnormal, dtype=int)
class_labels = ['Normal','Abnormal'] # label 0 = Normal; 1 = Abnormal

In [ ]:
# analysing the prediction (with 95% confidence interval)
analysis_1fold_95CT(true_classes,class_labels,test_prediction_5fold,predicted_classes_5fold,nsample=5000)

### Analysing the 5-fold prediction on the test set (in the unit of CT scans).

In [6]:
# load predictions from the DataFrame in the unit of CT scans
df_image_performance = get_df_image_performance(df,op)

In [7]:
# Get most likely class from the prodictions
test_prediction_avg_ct = df_image_performance.img_test_prediction_avg_5fold
predicted_classes_avg_ct = np.array(df_image_performance.img_predicted_classes_avg_5fold)
predicted_classes_ct = np.array(df_image_performance.img_predicted_classes_5fold, dtype=int)
# load ground truth from the DataFrame in the unit of CT scans
true_classes_ct = np.array(df_image_performance.abnormal, dtype=int)
class_labels_ct = ['Normal','Abnormal'] # label 0 = Normal; 1 = Abnormal

In [ ]:
# analysing the prediction 
analysis_1fold(true_classes_ct,class_labels_ct,test_prediction_avg_ct,predicted_classes_ct)